<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [5]</a>'.</span>

In [1]:
import pandas as pd
from dotenv import load_dotenv
import os
load_dotenv()

path_data = os.getenv("path_to_data")

In [2]:
df = pd.read_csv(f"../Daten/masterdata_2024.csv", parse_dates=True, index_col = 1)

C:\Users\Dome Arbeit\AppData\Local\Temp\ipykernel_15608\3846792546.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(f"../Daten/masterdata_2024.csv", parse_dates=True, index_col = 1)


In [3]:
bacteria_cols = ['ecoli', 'entro']

bacteria_df = df[df[bacteria_cols].notna().any(axis=1)].copy()

weather_cols = df.select_dtypes(include='number').columns.difference(bacteria_cols)

weather_df = df[weather_cols]



In [4]:
def aggregate_data(time, df):
    aggregations = ["mean", "median"]
    selected = df[(df.index >= time - pd.Timedelta(days=3)) & (df.index < time)]
    agg = selected.agg(aggregations)

    # Flatten values and create column names
    flattened_values = []
    flattened_columns = []
    for agg_name in aggregations:
        for col in agg.columns:
            flattened_values.append(agg.loc[agg_name, col])
            flattened_columns.append(f"{agg_name}_{col}")
    
    # Return as a single-row DataFrame
    return pd.DataFrame([flattened_values], columns=flattened_columns)


<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [5]:
aggregated_rows = []

for idx, row in bacteria_df.iterrows():
    agg_row = aggregate_data(idx, weather_df)  # idx is the timestamp

    # Add metadata
    agg_row['timestamp'] = idx
    for col in bacteria_cols:
        if col in row:  # safety check
            agg_row[col] = row[col]
    
    aggregated_rows.append(agg_row)


TypeError: unsupported operand type(s) for -: 'int' and 'Timedelta'

In [ ]:
bacteria_df

In [ ]:
for idx, row in bacteria_df.iterrows():
    print(bacteria_df.iloc[idx,, row)

In [ ]:
# Combine all into final DataFrame
final_df = pd.concat(aggregated_rows, ignore_index=True)

# Reorder columns
cols = ['timestamp'] + bacteria_cols + [c for c in final_df.columns if c not in ['timestamp'] + bacteria_cols]
final_df = final_df[cols]
